## Exercise 1: Implementation of multi-layer NN

#### 1. Initialization

In [1]:
import numpy as np
import math

In [2]:
np.random.seed(666)

In [3]:
class Model:
    def __init__(self, W, B):
        self.W = W
        self.B = B

In [4]:
def initialize(K, D, D_i, D_o):
    W = [None] * (K+1)
    B = [None] * (K+1)

    #initialize first layer
    W[0] = np.random.normal(size=(D, D_i))
    B[0] = np.random.normal(size =(D,1))

    #hidden layers
    for layer in range(1,K):
        W[layer] = np.random.normal(0, np.sqrt(2 / D), size=(D, D)) #He initialization
        B[layer] = np.zeros((D,1))

    #last layer
    W[-1] = np.random.normal(size=(D_o, D))
    B[-1]= np.random.normal(size =(D_o,1))


    return W,B

# Number of hidden layers
K = 2
# Number of neurons per layer
D = 6
# Input layer dimension
D_i = 784
# Output layer dimension
D_o = 10

In [5]:
W, B = initialize(K, D, D_i, D_o)
model = Model(W, B)

#### 2. Activation functions

In [6]:
def relu(x):
    activation = x.clip(0.0)
    return activation

def relu_der(x):
    dx = np.array(x)
    dx[dx>0] = 1
    dx[dx<=0] = 0
    return dx

def sigmoid(x):
    activation = 1.0 / (1.0 + np.exp(-x))
    return activation

def sigmoid_der(x):
    sigma = 1.0 / (1.0 + np.exp(-x))
    dx = sigma * (1.0 - sigma)
    return dx

#### 3. Forward pass

In [24]:
def forward_pass(net_input, W, B):
 
    K = len(W) -1 # number of layers

    F = [None] * (K+1) # pre-activations at each layer
    H = [None] * (K+1) # activations

    H[0] = net_input

    print(f"F init len: {len(F)}")
    print(f"H init len: {len(H)}")
    print(f"Input shape: {net_input.shape}")
    # Run through the layers, calculating F[0...K-1] and H[1...K]
    for layer in range(K):

        print(f"Processing layer {layer}/{K}")
        print(f"W[{layer}] shape = {W[layer].shape}")
        print(f"B[{layer}] shape = {B[layer].shape}")


        F[layer] = B[layer] + W[layer] @ H[layer] 
        print(f"F[{layer}] shape = {F[layer].shape}")


        H[layer+1] = relu(F[layer])
        print(f"H[{layer+1}] shape = {H[layer].shape}")

    
    F[K] = B[K] + W[K] @ H[K] 

    net_output = F[K]
    print(f"net_output shape = {net_output.shape}")


    return net_output, F, H

#### 4. Cost

In [ ]:
def softmax(x):

    shifted_x = x - np.max(x, axis=-1, keepdims=True) # For numerical stability when f >> 0,preventing overflow of exp(x)
    
    exp_x = np.exp(shifted_x)
    probs = exp_x / np.sum(exp_x, axis=-1, keepdims=True)
    
    return probs


def cross_entropy_cost(net_output, y):
    I = y.shape[1]  # Number of data points if data points as columns
    probs = softmax(net_output)
    return np.sum(-1 * y * np.log(probs))/I

def d_cost_d_output(net_output, y):
    I = y.shape[1] # Number of data points if data points as columns
    probs = softmax(net_output)
    print(f"probs shape: {probs.shape}")
    out = 2*np.sum(probs - y)/I
    print(f"d_cost_d_output shape: {out.shape}")
    return out

#### 5. Backprop

In [33]:
def backward_pass(W, B, F, H, y):

    K = len(W)-1
    # We'll store the derivatives dl_dweights and dl_dbiases in lists as well
    dl_dW = [None] * (K+1)
    dl_dB = [None] * (K+1)
    # And we'll store the derivatives of the cost wrt the activation and preactivations in lists
    dl_df = [None] * (K+1)
    dl_dh = [None] * (K+1)
    # Again for convenience we'll stick with the convention that H[0] is the net input and F[k] in the net output

    print("Backprop: ------------------------------")
    # Compute derivatives of the cost wrt the network output
    dl_df[K] = np.array(d_cost_d_output(F[K].T,y))
    print(f"dl_df[{K}] shape: {dl_df[K].shape}")

    # Change the range to the correct order in which the layers should be procesed.
    layer_range = np.flip(range(K+1))

    for layer in layer_range:
        # Calculate the derivatives of the cost wrt the biases at layer from dl_df[layer].
        dl_dB[layer] = np.sum(dl_df[layer], axis=1, keepdims=True) 
        print(f"dl_dB[{layer}] shape: {dl_dB[layer].shape}")


        # Calculate the derivatives of the cost wrt the weights at layer from dl_df[layer] and H[layer] 
        dl_dW[layer] = dl_df[layer] @ H[layer].T
        print(f"dl_dW[{layer}] shape: {dl_dW[layer].shape}")


        # Calculate the derivatives of cost wrt activations from weight and derivatives of next preactivations 
        dl_dh[layer] =  W[layer].T @ dl_df[layer]
        print(f"dl_dh[{layer}] shape: {dl_dh[layer].shape}")


        if layer > 0:
            # Calculate derivatives of the cost wrt pre-activation f 
            dl_df[layer-1] = relu_der(F[layer-1]) * dl_dh[layer] 
            print(f"dl_df[{layer-1}] shape: {dl_df[layer-1].shape}")

            
    return dl_dW, dl_dB

#### 6. Step & parameter update

In [10]:
def update_parameters(W, B, dW, dB, lr):
    W_new = W - lr*dW
    B_new = B - lr*dB
    return W_new, B_new

#### 7. Predict

In [11]:
def predict(model, x, y):
    res_logits = forward_pass(x, model.W, model.B)
    res = softmax(res_logits)

    # binarize
    res_binary = np.where(res == np.max(res), 1,0)
    acc = accuracy(y, res_binary)

    cost = cross_entropy_cost(res_logits, y)
    
    return res, acc, cost


def accuracy(y_true, y_pred):
    # having array of 1x10 vectors for true and pred, binary
    y_true_ind = np.argmax(y_true, axis=1)
    y_pred_ind = np.argmax(y_pred, axis=1)
    acc = np.mean(y_true_ind == y_pred_ind)

    return acc
    

In [12]:
def performance(y, preds):
    return 0

#### 8. Mini-batch

In [13]:
def random_mini_batches(x,y, size):
    assert len(x) == len(y)
    p = np.random.permutation(len(x))
    # x[p], y[p] are permuted arrays

    num_batches = math.ceil(len(x)/size)


    x_batched = np.array_split(x[p], num_batches)
    y_batched = np.array_split(y[p], num_batches)

    return x_batched, y_batched



#### 9. Model Training

In [14]:
# import imageio

# test = imageio.imread("MNIST/Train/4/0001.png")
# test = np.array(test)


In [16]:
from utils import load_mnist
X_train, Y_train, X_test, Y_test = load_mnist()

In [26]:
def train_model(X_train, Y_train, X_test, Y_test, model, epochs, lr, batch_size):

    X_batches, Y_batches = random_mini_batches(X_train, Y_train, batch_size)

    for epoch in range(epochs):

        for X_batch, Y_batch in zip(X_batches, Y_batches):
            # Forward pass
            net_out, F, H = forward_pass(X_batch.T, model.W, model.B)

            # Loss calculation
            loss = cross_entropy_cost(net_out, Y_batch.T)

            # Backward pass
            dl_dW, dl_dB = backward_pass(model.W, model.B, F, H, Y_batch)

            # Update parameters layer by layer
            for k in range(len(model.W)):
                model.W[k], model.B[k] = update_parameters(
                    model.W[k], model.B[k],
                    dl_dW[k], dl_dB[k],
                    lr
                )

        # Evaluate after each epoch
        _, train_acc, train_cost = predict(model, X_train.T, Y_train.T)
        _, test_acc, test_cost = predict(model, X_test.T, Y_test.T)

        print(f"Epoch {epoch+1}/{epochs} "
              f"| Train acc: {train_acc:.4f} "
              f"| Test acc: {test_acc:.4f} "
              f"| Train cost: {train_cost:.4f}"
              f"| Test cost: {test_cost:.4f}")

        


In [34]:
train_model(X_train, Y_train, X_test, Y_test, model, 5, 0.001, 32)

F init len: 3
H init len: 3
Input shape: (784, 32)
Processing layer 0/2
W[0] shape = (6, 784)
B[0] shape = (6, 1)
F[0] shape = (6, 32)
H[1] shape = (784, 32)
Processing layer 1/2
W[1] shape = (6, 6)
B[1] shape = (6, 1)
F[1] shape = (6, 32)
H[2] shape = (6, 32)
net_output shape = (10, 32)
Backprop: ------------------------------
dl_df[2] shape: ()


AxisError: axis 1 is out of bounds for array of dimension 0

In [30]:
len(model.W)

3